In [ ]:
from werkzeug.wrappers import Request, Response
from flask import Flask, render_template, request, url_for, flash, redirect
from flask import Flask
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import pandas as pd
import numpy as np
import os,cv2,imutils
import numpy as np
from scipy.spatial import distance
import cv2 
from flask import Flask, render_template, request, url_for, flash, redirect

app = Flask(__name__)

class DETECTION:
    def __init__(self):
        self.Labels=open("./coco.names",'r').read().split("\n")
        self.Network=cv2.dnn.readNetFromDarknet("./yolov3.cfg", "./yolov3.weights")
        self.UCOL=[self.Network.getLayerNames()[i-1] for i in self.Network.getUnconnectedOutLayers()]
    def Detect_People(self,frame,person_id=0):
        H,W=frame.shape[:2]
        Blob=cv2.dnn.blobFromImage(frame,1/255.0,(416,416),swapRB=True,crop=False)
        self.Network.setInput(Blob)
        Output_Layers=self.Network.forward(self.UCOL)
        BOXES,CONFIDENCES,CENTROIDS=[],[],[]
        for outputs in Output_Layers:
            for detected in outputs:
                scores = detected[5:]
                classID = np.argmax(scores)
                confidence = scores[classID]
                if classID==person_id and confidence>0.3:
                    BOX=detected[0:4]*np.array([W,H,W,H])
                    (X_CENTER,Y_CENTER,Width,Height)=BOX.astype("int")
                    X=(X_CENTER-(Width/2))
                    Y=(Y_CENTER-(Height/2))
                    BOXES.append([int(X),int(Y),int(Width),int(Height)])
                    CENTROIDS.append((X_CENTER,Y_CENTER))
                    CONFIDENCES.append(float(confidence))
        idx=cv2.dnn.NMSBoxes(BOXES, CONFIDENCES, 0.3, 0.3)
        result=[]
        if len(idx)>0:
            for i in idx.flatten():
                x,y=BOXES[i][0],BOXES[i][1]
                w,h=BOXES[i][2],BOXES[i][3]
                r=(CONFIDENCES[i],(x,y,x+w,y+h),CENTROIDS[i])
                result.append(r)
        return result
    def VIDEO_STREAM(self,video):
        V=cv2.VideoCapture(video)
        print("START DETECTING")
        while(True):
            _,frame=V.read()
            frame=imutils.resize(frame,width=1280,height=720)
            RESULT=self.Detect_People(frame,self.Labels.index("person"))
            voilate=set()
            try:
                if len(RESULT)>=2:
                    centroids=[i[2] for i in RESULT]
                    dists=distance.cdist(centroids,centroids,metric="euclidean")
                    for i in range(0,dists.shape[0]):
                        for j in range(i+1,dists.shape[1]):
                            if dists[i,j]<150:
                                voilate.add(i)
                                voilate.add(j)
                for i,(prob,box,cent) in enumerate(RESULT):
                    x,y,end_x,end_y=box
                    x_cent,y_cent=cent
                    color=(0,255,0)
                    if i in voilate:
                         color=(0,0,255)
                    cv2.rectangle(frame,(x,y),(end_x,end_y),color,2)
                cv2.imshow("video",frame)
                if cv2.waitKey(1) == ord("q"):
                    break
            except:
                pass
    def DESTROY_WINDOW(self):
        cv2.destroyAllWindows() 
           

class FCAKE_MASK():
    def __init__(self):
        self.mymodel=load_model('./New_Face_MasK_Model.h5')
        self.face_cascade=cv2.CascadeClassifier('./haarcascade_frontalface_default.xml')
    def RUN(self,inp):
        cap=cv2.VideoCapture(inp) #201202_01_Oxford Shoppers_4k_048_Trim_cropped
        while cap.isOpened():
            _,img=cap.read()
            face=self.face_cascade.detectMultiScale(img,scaleFactor=1.1,minNeighbors=4)
            for(x,y,w,h) in face:
                face_img = img[y:y+h, x:x+w]
                face_img=face_img/255.0
                face_img=cv2.resize(face_img,(224,224))
                face_img=image.img_to_array(face_img)
                face_img=np.expand_dims(face_img,axis=0)
                pred=np.round(self.mymodel.predict(face_img)[0][0])
                if pred==0:
                    cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),3)
                    cv2.putText(img,'MASK',((x+w)//2,y+h+20),cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,0),3)
                else:
                    cv2.rectangle(img,(x,y),(x+w,y+h),(0,0,255),3)
                    cv2.putText(img,'NO MASK',((x+w)//2,y+h+20),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),3)
                #datet=str(datetime.datetime.now())
                cv2.putText(img,'No mask',(400,450),cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,255,255),1)
            cv2.imshow('img',img)
            if cv2.waitKey(1)==ord('q'):
                break
        cap.release()
        cv2.destroyAllWindows()


obj=FCAKE_MASK()
RUN=DETECTION()

def myfunforsocialdistanceselfcamera():
    RUN.VIDEO_STREAM(0) 
    RUN.DESTROY_WINDOW()

def new(data):
    RUN.VIDEO_STREAM(data) # https://192.168.253.23:8080/video
    RUN.DESTROY_WINDOW()
    
def socialdistanceVedio(data):
    RUN.VIDEO_STREAM(data) # https://192.168.253.23:8080/video
    RUN.DESTROY_WINDOW()
    
def myownCamera():
    obj.RUN(0)

def myFun(data):
    obj.RUN(data)

def myFun2(data):
    obj.RUN(data)

@app.route('/',methods=('GET', 'POST'))
def index():
   if request.method == "POST":
      myFun()
   return render_template('index.html')

@app.route('/maskDetection',methods=('GET', 'POST'))
def maskDetection():
    print("Entering in Mask Detection")
    if request.method == "POST":
        if request.form['submit1']=='Search By IP':
            ipAdd = request.form['ip_add']
            print("Searching By Ip")
            if ipAdd == '0':
                myownCamera()
            else:
                myFun(str(ipAdd))
                      # socialdistanceVedio(str(socialdistanceVedioipAdd))
        elif request.form['submit1']=='Search In Video':
            print("Searching By Video")
            f = request.files['filename']
            myFun2(str(f.filename))
        return render_template('maskDetection.html')
    else:
        return render_template('maskDetection.html')
    
@app.route('/socialDistance',methods=('GET', 'POST'))
def socialDistance():
    print("Entering in Social Distance")
    if request.method == "POST":
        if request.form['submit']=='Search By IP':
            print("Searching By Ip")
            ipAdd = request.form['ip_add']
            if ipAdd == '0':
                myfunforsocialdistanceselfcamera()
            else:
                new(str(ipAdd))
                #socialdistanceVedio(str(socialdistanceVedioipAdd))
                #myFun(str(ipAdd))
        elif request.form['submit']=='Search In Video':
            print("Searching By Video")
            f = request.files['filename']
            socialdistanceVedio(str(f.filename))
        return render_template('socialDistance.html')
    else:
        return render_template('socialDistance.html')

@app.route('/aboutus',methods=('GET', 'POST'))
def aboutus():
   if request.method == "POST":
      myFun()
   return render_template('aboutus.html')

@app.route('/contactus',methods=('GET', 'POST'))
def contactus():
   if request.method == "POST":
      myFun()
   return render_template('contactus.html')

@app.route('/login',methods=('GET', 'POST'))
def login():
   import sqlite3 as sql
   con = sql.connect("./project1.db")
   if request.method == "POST":
      if request.form['submit'] == 'Sign In':
         username = request.form['username']
         password = request.form['password']
         cur = con.cursor()
         statement = f"SELECT email from users WHERE email='{username}' AND password = '{password}';"
         cur.execute(statement)
         if not cur.fetchone():  # An empty result evaluates to False.
            return render_template('login.html')
         else:
            return render_template('admin.html')
   else:
      return render_template('login.html')

@app.route('/signup',methods=('GET', 'POST'))
def signup():
   import sqlite3
   conn = sqlite3.connect('./project1.db')
   if request.method == "POST":
      if request.form['register'] == 'Register':
         name = request.form['name']
         email = request.form['email']
         password1 = request.form['password']
         password2 = request.form['confirmpassword']
         if password1 == password2:
            conn.execute("INSERT INTO users (name,email,password) VALUES ('"+name+"','"+email+"', '"+password1+"')")
            conn.commit()
            print("SignUp successfully")
            conn.close()
            return render_template('login.html')
         else:
            print('Please enter valid password.')
            return render_template('signup.html')
   else:
      return render_template('signup.html')

@app.route('/admin',methods=('GET', 'POST'))
def admin():
   return render_template('admin.html')

if __name__ == '__main__':
    from werkzeug.serving import run_simple
    run_simple('localhost', 9000, app)

 * Running on http://localhost:9000/ (Press CTRL+C to quit)
127.0.0.1 - - [18/Mar/2023 20:50:07] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [18/Mar/2023 20:50:07] "GET /static/style.css HTTP/1.1" 200 -
127.0.0.1 - - [18/Mar/2023 20:50:08] "GET /static/bg.jpg HTTP/1.1" 200 -
127.0.0.1 - - [18/Mar/2023 20:50:08] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [18/Mar/2023 20:50:21] "GET /aboutus HTTP/1.1" 200 -
127.0.0.1 - - [18/Mar/2023 20:50:21] "GET /static/img1.jpg HTTP/1.1" 200 -
127.0.0.1 - - [18/Mar/2023 20:50:30] "GET /login HTTP/1.1" 200 -
127.0.0.1 - - [18/Mar/2023 20:50:39] "POST /login HTTP/1.1" 200 -
127.0.0.1 - - [18/Mar/2023 20:50:42] "GET /signup HTTP/1.1" 200 -
127.0.0.1 - - [18/Mar/2023 20:51:03] "POST /signup HTTP/1.1" 200 -


SignUp successfully


127.0.0.1 - - [18/Mar/2023 20:51:20] "POST /signup HTTP/1.1" 400 -
127.0.0.1 - - [18/Mar/2023 20:51:36] "POST /signup HTTP/1.1" 400 -
127.0.0.1 - - [18/Mar/2023 20:51:46] "POST /signup HTTP/1.1" 400 -
127.0.0.1 - - [18/Mar/2023 20:51:53] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [18/Mar/2023 20:52:00] "GET /login HTTP/1.1" 200 -
127.0.0.1 - - [18/Mar/2023 20:52:12] "POST /login HTTP/1.1" 200 -
127.0.0.1 - - [18/Mar/2023 20:52:16] "GET /maskDetection HTTP/1.1" 200 -


Entering in Mask Detection
Entering in Mask Detection
Searching By Video
1/1 [==============================] - 0s 112ms/step


127.0.0.1 - - [18/Mar/2023 20:52:51] "POST /maskDetection HTTP/1.1" 200 -
127.0.0.1 - - [18/Mar/2023 20:52:58] "GET /maskDetection HTTP/1.1" 200 -


Entering in Mask Detection


127.0.0.1 - - [18/Mar/2023 20:53:00] "GET /socialDistance HTTP/1.1" 200 -


Entering in Social Distance
Entering in Social Distance
Searching By Video
START DETECTING


127.0.0.1 - - [18/Mar/2023 20:53:29] "POST /socialDistance HTTP/1.1" 200 -
